In [1]:
!pip install pyfastx obonet protein-bert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 903.2/903.2 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [2]:
import pyfastx
import obonet
import pandas as pd

from tensorflow import keras
from sklearn.model_selection import train_test_split
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

In [3]:
![ ! -f train_sequences.fasta ] && cp ../input/cafa-5-protein-function-prediction/Train/train_sequences.fasta .
fa = pyfastx.Fasta('train_sequences.fasta')
all_seqs = {s.name: s.seq for s in fa}

In [4]:
url = '../input/cafa-5-protein-function-prediction/Train/go-basic.obo'
graph = obonet.read_obo(url)
all_terms = list(graph.nodes())
all_labels = list(range(len(all_terms)))
term_to_label = {term: label for term, label in zip(all_terms, all_labels)}

In [25]:
train = pd.read_csv('../input/cafa-5-protein-function-prediction/Train/train_terms.tsv', sep='\t')

# Filter out terms with only 1 sample (since they cannot be splitted into train and validation sets)
train = train.groupby('term').filter(lambda x: x['EntryID'].count() >= 2)

train['text'] = train['EntryID'].map(lambda x: all_seqs[x])
train['label'] = train['term'].map(lambda x: term_to_label[x])

In [28]:
![ ! -f epoch_92400_sample_23500000.pkl ] && wget ftp://ftp.cs.huji.ac.il/users/nadavb/protein_bert/epoch_92400_sample_23500000.pkl

OUTPUT_TYPE = OutputType(False, 'categorical')
UNIQUE_LABELS = all_labels
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

train_set, valid_set = train_test_split(train, stratify = train['label'], test_size = 0.1, random_state = 0)

pretrained_model_generator, input_encoder = load_pretrained_model(
    local_model_dump_dir = '.',
    local_model_dump_file_name = 'epoch_92400_sample_23500000.pkl'
)

model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
        get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience = 1, factor = 0.25, min_lr = 1e-05, verbose = 1),
    keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
]

finetune(model_generator, input_encoder, OUTPUT_SPEC, train_set['text'], train_set['label'], valid_set['text'], valid_set['label'], \
        seq_len = 512, batch_size = 32, max_epochs_per_stage = 40, lr = 1e-04, begin_with_frozen_pretrained_layers = True, \
        lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = 1e-05, callbacks = training_callbacks)


[2023_04_22-02:14:33] Training set: Filtered out 3848 of 4823218 (0.1%) records of lengths exceeding 8190.


KeyboardInterrupt: 